In [2]:
import pandas as pd
import numpy as np
from os.path import join
import os
import urllib.parse
import urllib.request
from Bio.UniProt.GOA import _gpa11iterator
import gzip
import pickle

import warnings
warnings.filterwarnings('ignore')

In [3]:
import warnings
from rdkit import RDLogger

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        message=".*Chem.MolFromInchi.*")
warnings.filterwarnings("ignore", category=UserWarning,
                        message=".*SettingWithCopyWarning.*")
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# Suppress RDKit warnings
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

## 1. Extracting  UniProt IDs with transporter GO Terms:

In [3]:
df = pd.read_pickle(join("..", "..", "data", "GOA", "go_terms", "df_GO_with_substrates.pkl"))
transporter_go_terms = list(set(df["GO ID"]))

In [4]:
print(len(transporter_go_terms))

1904


The file "goa_uniprot_all.gpa.gz" needs to be downloaded from http://current.geneontology.org/annotations/index.html and stored in the folder "data/GOA/"  
Current file 12620552193 Apr 16 20:05 goa_uniprot_all.gpa.gz (2024)
To get the above file, need to:  
(base) miservilla@sahu:~$ ftp ftp://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/


In [5]:
filename = join("..", "..", "data", "GOA", "goa_uniprot_all.gpa.gz")

The code processes a large file of GO annotations, extracts relevant information from UniProt file for a specific subset of GO terms (transporter-related terms), and saves the resulting DataFrame in batches of 1 million annotations. The resulting DataFrame is saved as a pickle file for later use. Use extract_UniProt.py for tmux use.

In [ ]:
# Initialize variables
run = 0
batch_size = 10**6
df_GO_UID = pd.DataFrame(
    columns=["Uniprot ID", "GO Term", 'ECO_Evidence_code'])

overall_count = 0
continuing = False

with gzip.open(filename, 'rt') as fp:
    for annotation in _gpa11iterator(fp):
        overall_count += 1
        if overall_count >= run * batch_size and overall_count < (run + 1) * batch_size:
            # Extract data
            UID = annotation['DB_Object_ID']
            GO_ID = annotation['GO_ID']
            ECO_Evidence_code = annotation["ECO_Evidence_code"]
            if GO_ID in transporter_go_terms:
                df_GO_UID = pd.concat([df_GO_UID, pd.DataFrame([{
                    "Uniprot ID": UID,
                    "GO Term": GO_ID,
                    'ECO_Evidence_code': ECO_Evidence_code
                }])], ignore_index=True)
        else:
            # Save the current batch to a pickle file
            df_GO_UID.to_pickle(
                join("..", "..", "data", "GOA", "GO_UID_mapping", f"df_GO_UID_part_{run}.pkl"))
            # Reset the DataFrame for the next batch
            df_GO_UID = pd.DataFrame(
                columns=["Uniprot ID", "GO Term", 'ECO_Evidence_code'])
            run += 1

            # Make sure to include the current annotation in the new batch if it's part of the next run
            if overall_count >= run * batch_size and overall_count < (run + 1) * batch_size:
                UID = annotation['DB_Object_ID']
                GO_ID = annotation['GO_ID']
                ECO_Evidence_code = annotation["ECO_Evidence_code"]
                if GO_ID in transporter_go_terms:
                    df_GO_UID = pd.concat([df_GO_UID, pd.DataFrame([{
                        "Uniprot ID": UID,
                        "GO Term": GO_ID,
                        'ECO_Evidence_code': ECO_Evidence_code
                    }])], ignore_index=True)

# Save the final batch if any annotations were processed in the last run
if not df_GO_UID.empty:
    df_GO_UID.to_pickle(join("..", "..", "data", "GOA",
                        "GO_UID_mapping", f"df_GO_UID_part_{run}.pkl"))

In [6]:
'''This is part of orginal code that was used to extract the GO terms from the 
GOA file. It was modified to fix append issues. It does not allow for multiple 
pkl files to be created.'''

# run = 0

# df_GO_UID = pd.DataFrame(columns = ["Uniprot ID", "GO Term", 'ECO_Evidence_code'])


# overall_count = 0
# continuing = False

# with gzip.open(filename, 'rt') as fp:
#     for annotation in _gpa11iterator(fp):                 
#         overall_count += 1
#         if overall_count >= run*10**6 and overall_count < (run+1)*10**6:
#             # Output annotated protein ID   
#             UID = annotation['DB_Object_ID']
#             GO_ID = annotation['GO_ID']
#             ECO_Evidence_code = annotation["ECO_Evidence_code"]
#             if GO_ID in transporter_go_terms:
#                 df_GO_UID = pd.concat([df_GO_UID, pd.DataFrame([{"Uniprot ID": UID, "GO Term": GO_ID,
#                                                                 'ECO_Evidence_code': ECO_Evidence_code}])], ignore_index=True)
#             # if GO_ID in transporter_go_terms:
#             #     df_GO_UID = df_GO_UID.append({"Uniprot ID" : UID, "GO Term" : GO_ID,
#             #                                  'ECO_Evidence_code' : ECO_Evidence_code}, ignore_index = True)
           
# df_GO_UID.to_pickle(join("..", "..", "data", "GOA", "GO_UID_mapping", "df_GO_UID_part_" + str(run) +".pkl"))

In [8]:
'''This is part of orginal code that was used to extract the GO terms from the 
GOA file. It was modified to fix append issues. It does not allow for multiple 
pkl files to be created.'''

# # Initialize variables
# run = 0
# df_GO_UID = pd.DataFrame(columns=["Uniprot ID", "GO Term", 'ECO_Evidence_code'])
# overall_count = 0

# # Process the file
# with gzip.open(filename, 'rt') as fp:
#     for annotation in _gpa11iterator(fp):
#         overall_count += 1
#         if overall_count >= run * 10**6 and overall_count < (run + 1) * 10**6:
#             # Output annotated protein ID
#             UID = annotation['DB_Object_ID']
#             GO_ID = annotation['GO_ID']
#             ECO_Evidence_code = annotation["ECO_Evidence_code"]
#             if GO_ID in transporter_go_terms:
#                 df_GO_UID = pd.concat([df_GO_UID, pd.DataFrame([{"Uniprot ID": UID, "GO Term": GO_ID,
#                                                                 'ECO_Evidence_code': ECO_Evidence_code}])], ignore_index=True)

# # Save the DataFrame to a pickle file
# output_path = join("..", "..", "data", "GOA", "GO_UID_mapping", f"df_GO_UID_part_{run}.pkl")
# df_GO_UID.to_pickle(output_path)

In [4]:
df_GO_UID = pd.DataFrame(columns = ["Uniprot ID", "GO Term", 'ECO_Evidence_code'])

for run in range(1205):
    try:
        with open(join("..", "..", "data", "GOA", "GO_UID_mapping", "df_GO_UID_part_" + str(run) +".pkl"), "rb") as fh:
            df_new = pickle.load(fh)
        #df_new = pd.read_pickle(join(CURRENT_DIR, "alex_data", "go_data", "GO_UID_mapping", "df_GO_UID_part_" + str(run) +".pkl"))
        df_GO_UID = pd.concat([df_GO_UID, df_new], ignore_index=True)
    except:
        pass

df_GO_UID.to_pickle(join("..", "..", "data", "GOA", "df_GO_UID.pkl"))

In [5]:
df_GO_UID = pd.read_pickle(join("..", "..", "data", "GOA", "df_GO_UID.pkl"))
len(df_GO_UID)
df_GO_UID

,Uniprot ID,GO Term,ECO_Evidence_code
0,B4IDW0,GO:0006886,ECO:0000265
1,B4HNS0,GO:0015574,ECO:0000265
2,B4HNS0,GO:0005355,ECO:0000265
3,B4HNS0,GO:1904659,ECO:0000265
4,B4HNS0,GO:0015771,ECO:0000265
...,...,...,...
63770916,URS0002687D2E_666,GO:0015797,ECO:0000256
63770917,URS000268C02B_676,GO:0015797,ECO:0000256
63770918,URS0002698160_1481663,GO:0015797,ECO:0000256
63770919,URS00026C5A7C_669,GO:0015797,ECO:0000256


## 2. Mapping the ECO codes to 16 different evidence categories:

Every entry in the GOA database has an evidence code

In [ ]:
'''Need to correct append error with code block below'''
# filename_eco_obo = join("..", "..", "data", "GOA", 'eco.obo')

# df = pd.DataFrame(columns = ["ECO ID"])
# df["parents"] = ""

# file1 = open(filename_eco_obo, 'r')
# Lines = file1.readlines()
# start = 0
# while start != -1:
#     start = Lines.index('[Term]\n')
#     Lines = Lines[start+1:]
#     try:
#         ECO_Term = Lines[: Lines.index('[Term]\n')]
   

#         ECO_list = []
#         for line in ECO_Term:
#             if "is_a" in line:
#                 ID = line.split("! ")[0][6:-1]
#                 if ID.find("{") != -1:
#                     ID = ID[:ID.find("{")-1]
#                 ECO_list.append(ID)
            
#     except:
#         start = -1
            
    
#     df = df.append({"ECO ID" :  ECO_Term[0][4:-1], "parents" : ECO_list}, ignore_index = True)
    
# df["Evidence codes"] = ""

# ECO_to_GAF = pd.read_csv(join("..", "..", "data", "GOA", 'ECO_to_GAF.tsv'), sep = "\t")
# ECO_to_GAF

## 2. Mapping the ECO codes to 16 different evidence categories (Now with 20 categories):
#  http://purl.obolibrary.org/obo/eco/gaf-eco-mapping-derived.txt

In [8]:
import pandas as pd
from os.path import join

filename_eco_obo = join("..", "..", "data", "GOA", 'eco.obo')

df = pd.DataFrame(columns=["ECO ID", "parents"])

file1 = open(filename_eco_obo, 'r')
Lines = file1.readlines()
start = 0

while start != -1:
    try:
        start = Lines.index('[Term]\n')
        Lines = Lines[start + 1:]
        ECO_Term = Lines[:Lines.index('[Term]\n')]

        ECO_list = []
        for line in ECO_Term:
            if "is_a" in line:
                ID = line.split("! ")[0][6:-1]
                if "{" in ID:
                    ID = ID.split("{")[0].strip()
                ECO_list.append(ID)

        new_row = pd.DataFrame(
            {"ECO ID": [ECO_Term[0][4:-1]], "parents": [ECO_list]})
        df = pd.concat([df, new_row], ignore_index=True)
    except ValueError:
        start = -1

df["Evidence codes"] = ""

ECO_to_GAF = pd.read_csv(
    join("..", "..", "data", "GOA", 'ECO_to_GAF.tsv'), sep="\t")

df.to_pickle(join("..", "..", "data", "GOA", "df_ECO.pkl"))

In [10]:
ECO_to_GAF

,ECO,Evidence,code default
0,ECO:0000278,EXP,NaN
1,ECO:0000288,EXP,NaN
2,ECO:0001043,EXP,NaN
3,ECO:0001146,EXP,NaN
4,ECO:0001147,EXP,NaN
...,...,...,...
1401,ECO:0007659,RCA,NaN
1402,ECO:0007666,RCA,NaN
1403,ECO:0000245,RCA,Default
1404,ECO:0006017,TAS,NaN


In [ ]:
# import pandas as pd
# from os.path import join

# # Define the file paths
# filename_eco_obo = join("..", "..", "data", "GOA", 'eco.obo')

# # Initialize lists to store data
# eco_ids = []
# eco_parents = []

# # Read the eco.obo file
# with open(filename_eco_obo, 'r') as file1:
#     lines = file1.readlines()

# # Initialize variables
# current_term = None
# current_parents = []

# # Process each line
# for line in lines:
#     line = line.strip()
#     if line == "[Term]":
#         # Save the current term and its parents before starting a new term
#         if current_term:
#             eco_ids.append(current_term)
#             eco_parents.append(current_parents)
#         # Reset for the next term
#         current_term = None
#         current_parents = []
#     elif line.startswith("id: ECO:"):
#         current_term = line[4:].strip()
#     elif "is_a" in line:
#         parent_id = line.split()[1].strip()
#         current_parents.append(parent_id)

# # Append the last term if it exists
# if current_term:
#     eco_ids.append(current_term)
#     eco_parents.append(current_parents)

# # Create the DataFrame
# df = pd.DataFrame({
#     "ECO ID": eco_ids,
#     "parents": eco_parents
# })

# # Add the Evidence codes column, initially empty
# df["Evidence codes"] = ""

# # Print the DataFrame to verify
# print(df.head())

# # Read the ECO to GAF mapping file
# ECO_to_GAF = pd.read_csv(join("..", "..", "data", "GOA", 'ECO_to_GAF.tsv'), sep="\t")

# # Print the mapping file to verify
# print(ECO_to_GAF.head())


In [ ]:
# import pandas as pd
# from os.path import join

# # Define the file paths
# filename_eco_obo = join("..", "..", "data", "GOA", 'eco.obo')

# # Initialize the DataFrame
# df = pd.DataFrame(columns=["ECO ID", "parents"])

# # Read the eco.obo file
# with open(filename_eco_obo, 'r') as file1:
#     lines = file1.readlines()

# eco_terms = []
# eco_parents = []
# current_term = None
# current_parents = []

# # Process each line
# for line in lines:
#     line = line.strip()
#     if line == "[Term]":
#         if current_term:
#             eco_terms.append(current_term)
#             eco_parents.append(current_parents)
#         current_term = None
#         current_parents = []
#     elif line.startswith("id: ECO:"):
#         current_term = line[4:]
#     elif "is_a" in line:
#         parent_id = line.split()[1]
#         current_parents.append(parent_id)

# # Append the last term
# if current_term:
#     eco_terms.append(current_term)
#     eco_parents.append(current_parents)

# # Create the DataFrame
# df = pd.DataFrame({
#     "ECO ID": eco_terms,
#     "parents": eco_parents
# })

# # Print the DataFrame to verify
# print(df.head())

# # Read the ECO to GAF mapping file
# ECO_to_GAF = pd.read_csv(join("..", "..", "data", "GOA", 'ECO_to_GAF.tsv'), sep="\t")

# # Print the mapping file to verify
# print(ECO_to_GAF.head())


In [11]:
for ind in df.index:
    ID = df["ECO ID"][ind]
    help_df = ECO_to_GAF.loc[ECO_to_GAF["ECO"] == ID]
    if len(help_df) > 0:
        df["Evidence codes"][ind] = [list(help_df["Evidence"])[0]]
        
df_label = df.loc[df["Evidence codes"] != ""]
df_label.head()

df_label.to_pickle(join("..", "..", "data", "GOA", "df_ECO_label.pkl"))

In [12]:
df_label["Evidence codes"] = [code[0] for code in df_label["Evidence codes"]]
df_label.drop(columns =["parents"], inplace = True)
df_label.rename(columns = {"ECO ID" : "ECO_Evidence_code"}, inplace = True)

In [13]:
df_GO_UID = df_GO_UID.merge(df_label, how = "left", on = "ECO_Evidence_code")
df_GO_UID.rename(columns = {"Evidence codes" : "evidence"}, inplace = True)
df_GO_UID

,Uniprot ID,GO Term,ECO_Evidence_code,evidence
0,B4IDW0,GO:0006886,ECO:0000265,IEA
1,B4HNS0,GO:0015574,ECO:0000265,IEA
2,B4HNS0,GO:0005355,ECO:0000265,IEA
3,B4HNS0,GO:1904659,ECO:0000265,IEA
4,B4HNS0,GO:0015771,ECO:0000265,IEA
...,...,...,...,...
63770916,URS0002687D2E_666,GO:0015797,ECO:0000256,IEA
63770917,URS000268C02B_676,GO:0015797,ECO:0000256,IEA
63770918,URS0002698160_1481663,GO:0015797,ECO:0000256,IEA
63770919,URS00026C5A7C_669,GO:0015797,ECO:0000256,IEA


In [14]:
evidence_codes = list(set(df_GO_UID["evidence"]))
for ev in evidence_codes:
    print("%s : %s" % (ev, len(df_GO_UID.loc[df_GO_UID["evidence"] == ev])))

EXP : 95
ISO : 10767
ISA : 167
IBA : 182990
IEA : 63540215
HMP : 4
RCA : 934
IEP : 70
TAS : 2294
IPI : 88
ISM : 512
IDA : 12169
IGI : 1182
ISS : 10438
IGC : 67
NAS : 1626
IMP : 6961
IC : 342


#### Extracting all data points with experimental evidence

In [15]:
exp_evidence = ["EXP","IDA","IPI","IMP","IGI","IEP", "HTP","HDA","HMP","HGI","HEP"]

df_EXP = df_GO_UID.loc[df_GO_UID["evidence"] == "EXP"]
df_IDA = df_GO_UID.loc[df_GO_UID["evidence"] == "IDA"]
df_IPI = df_GO_UID.loc[df_GO_UID["evidence"] == "IPI"]
df_IMP = df_GO_UID.loc[df_GO_UID["evidence"] == "IMP"]
df_IGI = df_GO_UID.loc[df_GO_UID["evidence"] == "IGI"]
df_IEP = df_GO_UID.loc[df_GO_UID["evidence"] == "IEP"]

df_exp = pd.concat([df_EXP, df_IDA, df_IPI, df_IMP, df_IGI, df_IEP], ignore_index = True)
df_exp.drop_duplicates(inplace = True)
len(df_exp)

16809

In [16]:
df_GO_UID = df_exp.copy()

In [17]:
Uniprot_IDs = list(set(df_GO_UID["Uniprot ID"]))
print(len(Uniprot_IDs))

df_GO_UID.to_pickle(join("..", "..", "data", "GOA", "df_GO_UID_Transporter.pkl"))

7695


## 3. Mapping GO Terms to metabolite IDs:

In [18]:
df_GO_metabolite = pd.read_pickle(join("..", "..", "data", "GOA", "go_terms", "GO_terms_with_sub_IDs.pkl"))
df_GO_metabolite.head()

,GO ID,Definition,Name,Namespace,substrate,KEGG ID,PubChem CID,InChI,ChEBI
0,GO:0000006,"""Enables the transfer of zinc ions (Zn2+) from...",high-affinity zinc transmembrane transporter a...,molecular_function,zinc,NaN,23994.0,InChI=1S/Zn,CHEBI:30185
1,GO:0000007,"""Enables the transfer of a solute or solutes f...",low-affinity zinc ion transmembrane transporte...,molecular_function,zinc ion,C00038,NaN,NaN,CHEBI:29105
6,GO:0000064,"""Enables the transfer of L-ornithine from one ...",L-ornithine transmembrane transporter activity,molecular_function,l-ornithine,C00077,NaN,NaN,CHEBI:15729
7,GO:0000095,"""Enables the transfer of S-adenosylmethionine ...",S-adenosyl-L-methionine transmembrane transpor...,molecular_function,s-adenosyl-l-methionine,C00019,NaN,NaN,CHEBI:67040
11,GO:0000102,"""Enables the transfer of L-methionine from one...",L-methionine secondary active transmembrane tr...,molecular_function,l-methionine,C00073,NaN,NaN,CHEBI:16643


In [ ]:
# df_UID_MID = pd.DataFrame(columns =["Uniprot ID", "molecule ID"])

# for ind in df_GO_UID.index:
#     if ind >= -1:
#         GO_ID = df_GO_UID["GO Term"][ind]
#         UID = df_GO_UID["Uniprot ID"][ind]
#         met_IDs = list(df_GO_metabolite["ChEBI"].loc[df_GO_metabolite["GO ID"] == GO_ID])
#         for met_ID in met_IDs:
#             df_UID_MID = df_UID_MID.append({"Uniprot ID" : UID, "molecule ID" : met_ID}, ignore_index = True)
        
# df_UID_MID.drop_duplicates(inplace = True)
# Uniprot_IDs = list(set(df_UID_MID["Uniprot ID"]))
# print(len(Uniprot_IDs), len(list(set(df_UID_MID["molecule ID"]))))

# df_UID_MID

In [20]:
'''Fix append issue with code block below'''

df_UID_MID = pd.DataFrame(columns=["Uniprot ID", "molecule ID"])

rows = []  # Initialize an empty list to collect rows

for ind in df_GO_UID.index:
    if ind >= -1:
        GO_ID = df_GO_UID["GO Term"][ind]
        UID = df_GO_UID["Uniprot ID"][ind]
        met_IDs = list(
            df_GO_metabolite["ChEBI"].loc[df_GO_metabolite["GO ID"] == GO_ID])
        for met_ID in met_IDs:
            # Collect the row data
            rows.append({"Uniprot ID": UID, "molecule ID": met_ID})

# Convert the list of rows to a DataFrame
df_new_rows = pd.DataFrame(rows)

# Concatenate the new rows DataFrame to the existing DataFrame
df_UID_MID = pd.concat([df_UID_MID, df_new_rows], ignore_index=True)

df_UID_MID.drop_duplicates(inplace=True)
Uniprot_IDs = list(set(df_UID_MID["Uniprot ID"]))
print(len(Uniprot_IDs), len(list(set(df_UID_MID["molecule ID"]))))

df_UID_MID

3737 283


,Uniprot ID,molecule ID
0,Q9BXI2,CHEBI:15729
1,P10844,CHEBI:16541
2,Q4Q9Y4,CHEBI:18420
3,O14678,CHEBI:28911
4,Q9UBX3,CHEBI:35692
...,...,...
7516,A0A1D8PSH1,CHEBI:18133
7518,P12336,CHEBI:28757
7520,Q84W56,CHEBI:16411
7521,Q9CAT6,CHEBI:18127


## 4. Mapping UniProt IDs to amino acid sequences:

In [21]:
f = open(join("..", "..", "data", "GOA", "UNIPROT_IDs.txt"),"w") 

Uniprot_IDs = list(set(df_UID_MID["Uniprot ID"]))
for ID in list(set(Uniprot_IDs)):
    f.write(str(ID) + "\n")
f.close()

Using the Uniprot mapping service (https://www.uniprot.org/id-mapping) to map Uniprot IDs to sequences:

In [24]:
UNIPROT_df = pd.read_csv(join("..", "..", "data", "GOA", "Uniprot_results.csv"), sep = "\t")
UNIPROT_df.drop(columns = ["Entry"], inplace = True)

In [25]:
df_UID_MID = df_UID_MID.merge(UNIPROT_df, how = "left", on = "Uniprot ID")

In [26]:
df_UID_MID.to_pickle(join("..", "..", "data", "GOA", "GOA_Transporter.pkl"))

In [27]:
df_UID_MID

,Uniprot ID,molecule ID,Sequence
0,Q9BXI2,CHEBI:15729,MKSGPGIQAAIDLTAGAAGGTACVLTGQPFDTIKVKMQTFPDLYKG...
1,P10844,CHEBI:16541,MPVTINNFNYNDPIDNNNIIMMEPPFARGTGRYYKAFKITDRIWII...
2,Q4Q9Y4,CHEBI:18420,MMHSKLDGVTPGVAEAHQLSPLVARQRYLTVNRSGGSSWFVRKDVM...
3,O14678,CHEBI:28911,MAVAGPAPGAGARPRLDLQFLQRFLQILKVLFPSWSSQNALMFLTL...
4,Q9UBX3,CHEBI:35692,MAAEARVSRWYFGGLASCGAACCTHPLDLLKVHLQTQQEVKLRMTG...
...,...,...,...
5322,A0A1D8PSH1,CHEBI:18133,MSSVSSENNSGLFGTDVYDETKENKPKYEHEEGLEFGSDFDFDGEF...
5323,P12336,CHEBI:28757,MSEDKITGTLAFTVFTAVLGSFQFGYDIGVINAPQEVIISHYRHVL...
5324,Q84W56,CHEBI:16411,MMKPASLQGFSSHASSSIYSDVRRPATTPSKMAAFSALSLCPYTFT...
5325,Q9CAT6,CHEBI:18127,MEPSKQEVPKLMETPPNISNDSSATEKGEATRQQQLPNNRYALTVD...
